## Student: Lilit Beglaryan


## About data:
 retrieved from : https://www.kaggle.com/competitions/titanic/overview

The data is of some passengers of the well-known Titanic ship. While there was some randomness into who survived and who died in this tragic shipwrech, however it seems that some group of passengers were more likely than others to survive. 

The data contains information about passengers
The main goal is to analyze the relationship between the characteristics of passengers and their survival in teh Titanic's shipwrecking. The goal is to predict on unseen data during the test stage. The description of the variables is given bellow:

 Survival -- 0=No  ,  1=Yes    (Target to be predicted)

* PassengerId -- an assigned identifier to each passenger
* Pclass -- (ticket class) 
1=Upper, 2=Middle, 3=Lower
* Name -- name,last name
* Sex -- male/female
* Age -- age in years
* Sibsp -- # of parents / children aboard the Titanic
* Parch -- 	# of parents / children aboard the Titanic

* Ticket -- Ticket number
* Fare	-- Passenger fare
* Cabin	-- Cabin number	

* Embarked -- Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton


In [1]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix

In [4]:
import statsmodels.api as sm

In [5]:

from sklearn import preprocessing

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import scikit_posthocs as sp
from scipy import stats
pd.options.display.float_format = '{:,.4f}'.format

In [7]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")



In [8]:
print(train.shape)
print(test.shape)

(891, 12)
(418, 11)


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [10]:
train.dropna(subset = ["Age"], inplace = True)

In [11]:
print(train.Cabin.isna().sum()) #too many NaN values, so drop
train.drop(columns="Cabin",inplace=True)


529


In [12]:
train.dropna(axis=0, subset="Embarked", inplace=True) #drop the 2rows where leaving port(Embarked) is missing
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Survived     712 non-null    int64  
 2   Pclass       712 non-null    int64  
 3   Name         712 non-null    object 
 4   Sex          712 non-null    object 
 5   Age          712 non-null    float64
 6   SibSp        712 non-null    int64  
 7   Parch        712 non-null    int64  
 8   Ticket       712 non-null    object 
 9   Fare         712 non-null    float64
 10  Embarked     712 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 66.8+ KB


In [13]:
### TRaining stage
train.columns
train.drop(["Name", "Ticket", "Embarked","PassengerId"], axis = 1, inplace = True)  #since the names,ids, departure ports and ticket names do not
# influence the survival


print(train.head())


   Survived  Pclass     Sex     Age  SibSp  Parch    Fare
0         0       3    male 22.0000      1      0  7.2500
1         1       1  female 38.0000      1      0 71.2833
2         1       3  female 26.0000      0      0  7.9250
3         1       1  female 35.0000      1      0 53.1000
4         0       3    male 35.0000      0      0  8.0500


In [14]:
train["Sex"] = pd.get_dummies(train['Sex'],drop_first = True) #since model fitting only works for float64 dtypes not strings
train.rename(columns = {'Sex':'IsMale'}, inplace = True)
train = pd.get_dummies(train, columns = ["Pclass"], drop_first=True)

print(train)

     Survived  IsMale     Age  SibSp  Parch    Fare  Pclass_2  Pclass_3
0           0       1 22.0000      1      0  7.2500         0         1
1           1       0 38.0000      1      0 71.2833         0         0
2           1       0 26.0000      0      0  7.9250         0         1
3           1       0 35.0000      1      0 53.1000         0         0
4           0       1 35.0000      0      0  8.0500         0         1
..        ...     ...     ...    ...    ...     ...       ...       ...
885         0       0 39.0000      0      5 29.1250         0         1
886         0       1 27.0000      0      0 13.0000         1         0
887         1       0 19.0000      0      0 30.0000         0         0
889         1       1 26.0000      0      0 30.0000         0         0
890         0       1 32.0000      0      0  7.7500         0         1

[712 rows x 8 columns]


In [15]:
y = train['Survived'].copy()

X = train.drop(["Survived"],axis = 1)


print(type(y))


<class 'pandas.core.series.Series'>


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X,y ,random_state=104, test_size=0.25, shuffle=True)

In [17]:
X_test= sm.add_constant(X_test)

In [18]:
X_train= sm.add_constant(X_train)

In [19]:
print(X_test.columns)

Index(['const', 'IsMale', 'Age', 'SibSp', 'Parch', 'Fare', 'Pclass_2',
       'Pclass_3'],
      dtype='object')


In [20]:
log_reg = sm.Logit(np.asarray(y_train), X_train).fit()

Optimization terminated successfully.
         Current function value: 0.459698
         Iterations 6


In [21]:
# printing the summary table
print(log_reg.summary())  

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  534
Model:                          Logit   Df Residuals:                      526
Method:                           MLE   Df Model:                            7
Date:                Tue, 21 Mar 2023   Pseudo R-squ.:                  0.3241
Time:                        16:27:28   Log-Likelihood:                -245.48
converged:                       True   LL-Null:                       -363.19
Covariance Type:            nonrobust   LLR p-value:                 3.510e-47
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.3885      0.588      7.466      0.000       3.236       5.541
IsMale        -2.5209      0.253     -9.982      0.000      -3.016      -2.026
Age           -0.0475      0.010     -4.921      0.0

In [ ]:
#so  drop the Parch, Fare columns since they are not significant for alpha=0.05

In [22]:
train.drop(["Parch","Fare"],axis = 1, inplace =True)

In [23]:
y = train['Survived'].copy()

X = train.drop(["Survived"],axis = 1)


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X,y ,random_state=104, test_size=0.25, shuffle=True)

In [25]:
X_train= sm.add_constant(X_train)


In [26]:
X_test=sm.add_constant(X_test)

In [27]:
#train again now only with the significant independent variables
model = sm.Logit(y_train, X_train).fit()

Optimization terminated successfully.
         Current function value: 0.460412
         Iterations 6


In [28]:
pred = model.predict(X_test)
pred = list(map(round,pred))

In [29]:
cm = confusion_matrix(y_test, pred) 
print ("Confusion Matrix : \n", cm) 

Confusion Matrix : 
 [[93 21]
 [11 53]]


In [30]:
# 
train["Survived"].value_counts()  # imbalanced data since much more 0s than 1s, 
# so pay attention to the recall and precision of 1s(the smallest set of class)



0    424
1    288
Name: Survived, dtype: int64

In [31]:
# accuracy score of the model
print('Test accuracy = ', accuracy_score(y_test, pred))  #not so descriptive of our model's performance since the labels are not balanced

Test accuracy =  0.8202247191011236


In [32]:
print('Precision Score = ', precision_score(y_test, pred))  #~~0.71 , 
# can be improved by resplitting the data and increasing the train data portion,
# or adding more synthetic data and bringing it to a balance

Precision Score =  0.7162162162162162


In [33]:
print('Recall= ', recall_score(y_test, pred))  # ~~0.82 , 
# can be improved by resplitting the data and increasing the train data portion,
# or adding more synthetic data and bringing to a balance

Recall=  0.828125


In [34]:
print('F1 Score= ', f1_score(y_test, pred))  # ~~0.77 , 
# can be improved by resplitting the data and increasing the train data portion, 
# or adding more synthetic data and bringing to a balance

F1 Score=  0.7681159420289855


In [35]:
print(model.summary())   #all variables are significant 

                           Logit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  534
Model:                          Logit   Df Residuals:                      528
Method:                           MLE   Df Model:                            5
Date:                Tue, 21 Mar 2023   Pseudo R-squ.:                  0.3230
Time:                        16:28:42   Log-Likelihood:                -245.86
converged:                       True   LL-Null:                       -363.19
Covariance Type:            nonrobust   LLR p-value:                 1.077e-48
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.5406      0.537      8.452      0.000       3.488       5.594
IsMale        -2.5079      0.248    -10.129      0.000      -2.993      -2.023
Age           -0.0480      0.010     -4.989      0.0

In [ ]:
#now trying to add more training data

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X,y ,random_state=104, test_size=0.1, shuffle=True)

In [37]:
X_train= sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

In [38]:
#train again now only with 0.9:0.1 ratio of train:test
model = sm.Logit(y_train, X_train).fit()

Optimization terminated successfully.
         Current function value: 0.447575
         Iterations 6


In [39]:
pred = model.predict(X_test)
pred = list(map(round,pred))

In [40]:
cm = confusion_matrix(y_test, pred) 
print ("Confusion Matrix : \n", cm) 

Confusion Matrix : 
 [[38 11]
 [ 4 19]]


In [41]:
# accuracy score of the model
print('Test accuracy = ', accuracy_score(y_test, pred))  #not so descriptive of our model's performance since the labels are not balanced

Test accuracy =  0.7916666666666666


In [42]:
print('Precision Score = ', precision_score(y_test, pred))  #~~0.63 , actually downgarded

Precision Score =  0.6333333333333333


In [43]:
print('f1 Score = ', f1_score(y_test, pred))  #~~0.72 , actually downgarded

f1 Score =  0.7169811320754716


In [44]:
print('recall Score = ', recall_score(y_test, pred))  #~~0.83 , approximately the same

recall Score =  0.8260869565217391


In [45]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  640
Model:                          Logit   Df Residuals:                      634
Method:                           MLE   Df Model:                            5
Date:                Tue, 21 Mar 2023   Pseudo R-squ.:                  0.3402
Time:                        16:29:19   Log-Likelihood:                -286.45
converged:                       True   LL-Null:                       -434.11
Covariance Type:            nonrobust   LLR p-value:                 1.010e-61
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.6188      0.489      9.437      0.000       3.659       5.578
IsMale        -2.6050      0.229    -11.397      0.000      -3.053      -2.157
Age           -0.0501      0.009     -5.706      0.000      -0.067      -0.033
SibSp         -0.4360      0.126     -3.452      0.001      -0.684      -0.188
Pclass_2      -1.4269      0.304     -4.689      0.000      -2.023      -0.830
Pclass_3      -2.7014      0.302     -8.956      0.000      -3.293      -2.110
==============================================================================
"""

In [48]:
#although not a good model but can estimate Rose's and Jack's survival rates
# based on the estimated coeeficients

In [49]:
np.exp(4.6188 + 1*-2.6050 + 20*-.0501 + 0 -2.7014 )/(1+np.exp(4.6188 + 1*-2.6050 + 20*-.0501 + 0 -2.7014 )) 
#Jack was 20y old,with no parents(all died) and siblings on the board, was a poor lad and traveled in the 3rd class
#  the survival rate is 0.15 which is way lower than the boundary 0.5, which means we can consider it as 0(meaning would not survive)
# which actually happened

0.15582845125815836

In [51]:
np.exp(4.6188 + 0 + 17*-.0501 + 1*-0.4360 )/(1+np.exp(4.6188 + 0 + 17*-.0501 + 1*-0.4360 )) 
#Rose was 17y old at the time of ship wrecking,with her fiancee on the board(SibSp =1), 
#  traveled in the 1st class
#  the survival rate is 0.9654804492375018 which is way higher than the boundary 0.5, 
# which means we can consider it as 1(meaning would survive)
# which actually happened

0.9654804492375018

In [ ]:
 ## adding Age square

In [52]:
X.head()

,IsMale,Age,SibSp,Pclass_2,Pclass_3
0,1,22.0000,1,0,1
1,0,38.0000,1,0,0
2,0,26.0000,0,0,1
3,0,35.0000,1,0,0
4,1,35.0000,0,0,1


In [53]:
X["Age_squared"] =X["Age"]**2

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X,y ,random_state=104, test_size=0.2)

In [55]:
X_train= sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

In [56]:
model = sm.Logit(y_train, X_train).fit()

Optimization terminated successfully.
         Current function value: 0.447429
         Iterations 6


In [57]:
pred = model.predict(X_test)
pred = list(map(round,pred))

In [58]:
print('Test accuracy = ', accuracy_score(y_test, pred)) #0.84, improved

Test accuracy =  0.8391608391608392


In [59]:
print('Precision Score = ', precision_score(y_test, pred)) # 0.74,improved

Precision Score =  0.7450980392156863


In [60]:
print('f1 Score = ', f1_score(y_test, pred)) #0.76,slightly improved

f1 Score =  0.7676767676767677


In [61]:
print('recall Score = ', recall_score(y_test, pred)) #0.79 recall worsened, while the others got improved

recall Score =  0.7916666666666666


In [62]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  569
Model:                          Logit   Df Residuals:                      562
Method:                           MLE   Df Model:                            6
Date:                Tue, 21 Mar 2023   Pseudo R-squ.:                  0.3429
Time:                        16:31:02   Log-Likelihood:                -254.59
converged:                       True   LL-Null:                       -387.41
Covariance Type:            nonrobust   LLR p-value:                 1.850e-54
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           5.5898      0.656      8.517      0.000       4.303       6.876
IsMale         -2.6167      0.246    -10.619      0.000      -3.100      -2.134
Age            -0.1146      0.027     -4.212      0.000      -0.168      -0.061
SibSp          -0.5572      0.140     -3.992      0.000      -0.831      -0.284
Pclass_2       -1.3554      0.320     -4.237      0.000      -1.982      -0.728
Pclass_3       -2.7228      0.321     -8.492      0.000      -3.351      -2.094
Age_squared     0.0010      0.000      2.555      0.011       0.000       0.002
===============================================================================
"""